# Ollama PDF RAG Notebook

## Import Libraries


In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [2]:
# Load PDF
local_path = "D:\Progect\AI_assistant\data\pdfs\sample\Методические рекомендации РТПП.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: D:\Progect\AI_assistant\data\pdfs\sample\Методические рекомендации РТПП.pdf


## Split text into chunks

In [3]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 15 chunks


## Create vector database

In [5]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [18]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [19]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Ты - AI-ассистент языковой модели. Твоя задача - сгенерировать 2
    разные версии заданного пользователем вопроса для поиска релевантных документов в
    векторной базе данных. Генерируя множественные перспективы на вопрос пользователя, твоя
    цель - помочь пользователю преодолеть некоторые ограничения основанного на расстоянии
    поиска по сходству. Предоставь эти альтернативные вопросы, разделенные новой строкой.
    Исходный вопрос: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [20]:
# RAG prompt template
template = """Отвечайте на вопрос ТОЛЬКО на основе следующего контекста:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [21]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [22]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [11]:
# Example 1
chat_with_pdf("О чем этот документ?")

Документ описывает методические рекомендации по разработке регламентов технологических процессов в горнодобывающей промышленности. 

В нем перечислены виды производственных процессов, на которые рекомендуется разрабатывать регламенты, а также требования к их содержанию.  Регламенты должны включать мероприятия по обеспечению безопасности при проведении всех видов горных работ, учитывая опасные факторы и специфику работы в шахтах и рудниках. 


In [17]:
# Example 2 - mistral
chat_with_pdf("О чем этот документ?")

 This document is about the methodological recommendations on the order of developing, approving and confirming Regulations of Technological Production Processes when conducting underground mining work. The title of this specific document is "Methodological Recommendations on the Procedure for Drafting, Approval and Confirmation of Regulations of Technological Production Processes during Underground Mining Work" (RD 06-627-03). It was approved by the Decision of the State Technical Supervision of Russia dated September 15, 2003, No. 108 and came into force on January 1, 2004 with the Decision of the State Technical Supervision of Russia dated December 9, 2003, No. 112. It is provided by ConsultantPlus (www.consultant.ru). The document specifies the guidelines for drafting and confirming regulations on various production processes during underground mining operations.

In [23]:
# Example 3 llama3.2
chat_with_pdf("О чем этот документ?")

Этот document является Методическими рекомендациями по промышленной безопасности в шахтах и рудниках.

## Clean up (optional)

In [27]:
# Optional: Clean up when done 
vector_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
